In [1]:
# choose your data set:
sim_data_path = 'Traffic_light_sim_only'
real_data_path = 'Traffic_light_real_only'
both_data_path = 'Traffic_light_both'

def get_validation_path(path):
    return path + '/test_images'

In [2]:
# choose your data set:
train_data_path = both_data_path

validation_data_path = get_validation_path(train_data_path)




from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

batch_size = 64

nb_epoch = 225
nb_classes = 4
image_shape = [64, 64, 3]

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=image_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# save best model only 
# NOTE: needs validation_data in fit_generator or validation_split in ImageDataGenerator, which isn't available
model_file_name = train_data_path + '_' + 'light_classifier_model.h5'
checkpointer = ModelCheckpoint(monitor='val_acc', filepath=model_file_name, save_best_only=True, verbose=1)

# reduce learning rate when loss doesn't improve much anymore (to pin-point the local minima)
# reduce_lr = ReduceLROnPlateau(monitor='val_acc', epsilon=0.03, factor=0.4, patience=4, min_lr=0.001, verbose=1)

# stop when there was no real improvement anymore (even after reducing the learning rate several times)
early_stop = EarlyStopping(monitor='val_acc', min_delta=0.01, patience=15, verbose=1)


# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.2,
                             fill_mode='nearest', rescale=1. / 255)

image_train_data_gen = datagen.flow_from_directory(train_data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

image_valid_data_gen = datagen.flow_from_directory(validation_data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

model.fit_generator(image_train_data_gen, nb_epoch=nb_epoch,
                    samples_per_epoch=image_train_data_gen.nb_sample,
                    validation_data=image_valid_data_gen,
                    nb_val_samples=image_valid_data_gen.nb_sample,
                    callbacks=[checkpointer, early_stop])

# model.save('light_classifier_model.h5')

Using TensorFlow backend.


Found 5016 images belonging to 4 classes.
Found 390 images belonging to 4 classes.
Epoch 1/225
5016/5016 [==============================] - 122s - loss: 1.1691 - acc: 0.5556 - val_loss: 1.1317 - val_acc: 0.4795
Epoch 2/225
5016/5016 [==============================] - 61s - loss: 0.7978 - acc: 0.6691 - val_loss: 1.1898 - val_acc: 0.4564
Epoch 3/225
5016/5016 [==============================] - 65s - loss: 0.6548 - acc: 0.7404 - val_loss: 1.2899 - val_acc: 0.4692
Epoch 4/225
5016/5016 [==============================] - 62s - loss: 0.5688 - acc: 0.7757 - val_loss: 0.6616 - val_acc: 0.6974
Epoch 5/225
5016/5016 [==============================] - 64s - loss: 0.5344 - acc: 0.7865 - val_loss: 0.7314 - val_acc: 0.6795
Epoch 6/225
5016/5016 [==============================] - 67s - loss: 0.4846 - acc: 0.8102 - val_loss: 0.6707 - val_acc: 0.7103
Epoch 7/225
5016/5016 [==============================] - 64s - loss: 0.4380 - acc: 0.8180 - val_loss: 0.6851 - val_acc: 0.6897
Epoch 8/225
5016/5016 [====

5016/5016 [==============================] - 62s - loss: 0.1776 - acc: 0.9276 - val_loss: 0.2792 - val_acc: 0.8385
Epoch 31/225
5016/5016 [==============================] - 65s - loss: 0.1726 - acc: 0.9306 - val_loss: 0.2327 - val_acc: 0.8744
Epoch 32/225
5016/5016 [==============================] - 64s - loss: 0.1900 - acc: 0.9234 - val_loss: 0.3330 - val_acc: 0.8615
Epoch 33/225
5016/5016 [==============================] - 64s - loss: 0.1701 - acc: 0.9336 - val_loss: 0.2683 - val_acc: 0.8641
Epoch 34/225
5016/5016 [==============================] - 64s - loss: 0.1516 - acc: 0.9408 - val_loss: 0.1870 - val_acc: 0.9179
Epoch 35/225
5016/5016 [==============================] - 63s - loss: 0.1577 - acc: 0.9402 - val_loss: 0.2496 - val_acc: 0.9026
Epoch 36/225
5016/5016 [==============================] - 62s - loss: 0.1434 - acc: 0.9446 - val_loss: 0.2232 - val_acc: 0.9000
Epoch 37/225
5016/5016 [==============================] - 62s - loss: 0.1531 - acc: 0.9396 - val_loss: 0.1841 - val_a

5016/5016 [==============================] - 60s - loss: 0.0965 - acc: 0.9657 - val_loss: 0.1047 - val_acc: 0.9641
Epoch 62/225
5016/5016 [==============================] - 60s - loss: 0.1011 - acc: 0.9651 - val_loss: 0.1028 - val_acc: 0.9667
Epoch 63/225
5016/5016 [==============================] - 60s - loss: 0.1100 - acc: 0.9639 - val_loss: 0.1347 - val_acc: 0.9641
Epoch 64/225
5016/5016 [==============================] - 63s - loss: 0.0994 - acc: 0.9649 - val_loss: 0.1892 - val_acc: 0.9333
Epoch 65/225
5016/5016 [==============================] - 61s - loss: 0.0981 - acc: 0.9659 - val_loss: 0.1225 - val_acc: 0.9667
Epoch 66/225
5016/5016 [==============================] - 60s - loss: 0.0952 - acc: 0.9651 - val_loss: 0.1707 - val_acc: 0.9410
Epoch 67/225
5016/5016 [==============================] - 61s - loss: 0.1670 - acc: 0.9402 - val_loss: 0.1172 - val_acc: 0.9538
Epoch 68/225
5016/5016 [==============================] - 60s - loss: 0.1069 - acc: 0.9621 - val_loss: 0.1567 - val_a

In [3]:
model.load_weights(filepath=model_file_name)

datagen = ImageDataGenerator(fill_mode='nearest', rescale=1. / 255)  # no randomization

data_path = sim_data_path
train_data_gen = datagen.flow_from_directory(data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])
valid_data_gen = datagen.flow_from_directory(get_validation_path(data_path), target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

print("Simulation Training loss and accuracy: ", model.evaluate_generator(train_data_gen, train_data_gen.nb_sample))
print("Simulation Validation loss and accuracy: ", model.evaluate_generator(valid_data_gen, valid_data_gen.nb_sample))


data_path = real_data_path
train_data_gen = datagen.flow_from_directory(data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])
valid_data_gen = datagen.flow_from_directory(get_validation_path(data_path), target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

print("RealWorld Training loss and accuracy: ", model.evaluate_generator(train_data_gen, train_data_gen.nb_sample))
print("RealWorld Validation loss and accuracy: ", model.evaluate_generator(valid_data_gen, valid_data_gen.nb_sample))

Found 2165 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
('Simulation Training loss and accuracy: ', [0.028399262228855911, 0.98752886836027709])
('Simulation Validation loss and accuracy: ', [0.028542196303606035, 0.98999999999999999])
Found 2851 images belonging to 4 classes.
Found 190 images belonging to 4 classes.
('RealWorld Training loss and accuracy: ', [0.043431120958298139, 0.98982813052234631])
('RealWorld Validation loss and accuracy: ', [0.092397904082348478, 0.98421052631578942])


In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 64, 64, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 64, 64, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 62, 62, 32)    9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 62, 62, 32)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________